In [1]:
import textwrap

PC1 = [28, 27, 22, 21, 25, 30, 4, 29, 0, 16, 20, 17, 23, 1, 10, 18, 3, 6, 2, 24, 19, 13, 15, 31, 8, 9, 14, 11, 5, 12]
PC2 = [24, 13, 15, 2 ,21, 12, 25, 10, 6 ,0 ,17 ,22, 4 ,3 ,1 ,23 ,11 ,14 ,28 ,20 ,9 ,27 ,16 ,19, 26, 7 ,5 ,18 ,8 ,2,9]
EXPANSION_TABLE = [16, 20, 7, 1, 17, 7 ,23 ,3, 16 ,9 ,10 ,18 ,12 ,2 ,11 ,13, 15, 4 ,20 ,15, 3 ,23 ,14, 3 ,11 ,0, 1, 20, 2, 20]
PERMUTATION_TABLE = [6, 0, 9, 15, 8 ,2 ,12, 14, 13, 22 ,18 ,5 ,4, 16, 19 ,21 ,23 ,11 ,17 ,1 ,7, 20, 3 ,10]

SBOX = [
[
    [0, 3, 1, 7, 4, 5, 2, 8, 6],
    [1, 5, 4, 6, 7, 3, 0, 2, 8],
    [7, 1, 2, 6, 5, 0, 8, 4, 3],
    [5, 7, 4, 0, 8, 2, 1, 6, 3]
],


[
    [3, 7, 5, 2, 6, 1, 8, 4],
    [5, 2, 0, 4, 8, 7, 1, 3],
    [6, 0, 7, 8, 1, 4, 5, 2],
    [6, 3, 1, 5, 8, 0, 7, 4]
],

[
    [4, 0, 3, 5, 2, 7, 6, 8],
    [5, 1, 4, 3, 2, 0, 8, 7],
    [0, 8, 6, 2, 5, 1, 4, 7],
    [7, 8, 1, 0, 3, 5, 4, 2]
],
    
[
    [6, 2, 8, 3, 5, 1, 4, 0],
    [4, 5, 2, 0, 3, 7, 6, 8],
    [5, 1, 4, 6, 8, 0, 2, 7],
    [8, 2, 1, 4, 7, 6, 3, 0]
],
    
[
    [4, 8, 7, 0, 1, 5, 3, 2],
    [5, 8, 0, 3, 7, 4, 6, 1],
    [3, 7, 1, 6, 8, 4, 5, 0],
    [2, 4, 6, 8, 0, 5, 3, 1]
],
    
[
    [1, 8, 6, 7, 4, 5, 0, 3],
    [0, 5, 3, 7, 1, 2, 6, 8],
    [4, 5, 2, 7, 1, 0, 6, 8],
    [3, 0, 7, 5, 6, 4, 8, 1]
]
]

def apply_PC1(pc1_table, keys_32bits):
    keys_30bits = ""
    for index in pc1_table:
        keys_30bits += keys_32bits[index] 
    return keys_30bits

def split30bits_to_three(keys_30bits):
    left, mid, right = keys_30bits[:10], keys_30bits[10:20],keys_30bits[20:]
    return left, mid, right

def shift(bits, numberofbits):
    shiftedbits = bits[numberofbits:] + bits[:numberofbits]
    return shiftedbits

def apply_PC2(pc2_table, keys_30bits):
    final = ""
    for index in pc2_table:
        final += keys_30bits[index]
    return final

def generate_keys(key_32bits):
    keys = []
    
    key_30bits = apply_PC1(PC1, key_32bits)
    left, mid, right = split30bits_to_three(key_30bits)
    
    for i in range(16):
        if i%2 == 0:
            left = shift(left, 1)
            mid = shift(mid, 1)
            right = shift(right, 1)
        else:
            left = shift(left, i%10)
            mid = shift(mid, i%10)
            right = shift(right, i%10)

        key = apply_PC2(PC2, left + mid + right)
        keys.append(key)
    
    return keys

def expansion24_to_30(expansion_table, bits24):
    bits48 = ""
    for index in expansion_table:
        bits48 += bits24[index-1]
    return bits48

def XOR(bits1,bits2):
    xor_result = ""
    for index in range(len(bits1)):
        if bits1[index] == bits2[index]: 
            xor_result += '0'
        else:
            xor_result += '1'
    return xor_result

def split30bits_in_5bits(XOR_30bits):
    return textwrap.wrap(XOR_30bits, 5)

def binary_to_decimal(binarybits):
    decimal = int(binarybits,2)
    return decimal

def decimal_to_binary(decimal):
    binary4bits = bin(decimal)[2:].zfill(4)
    return binary4bits

def sbox_result(sboxcount, odd, even):
    sbox_value = SBOX[sboxcount][binary_to_decimal(odd)][binary_to_decimal(even)]
    return decimal_to_binary(sbox_value)

def F(pre24bits, key30bits):
    final = ''

    out_bit30 = expansion24_to_30(EXPANSION_TABLE, pre24bits)
    xor_bits30 = XOR(out_bit30, key30bits)

    for cnt, part in enumerate(split30bits_in_5bits(xor_bits30)):
        even_bits = (part[0] + part[2] + part[4])
        odd_bits = (part[1] + part[3])
        result = sbox_result(cnt, odd_bits, even_bits)
        final += result
    
    final = permutation(PERMUTATION_TABLE,final)
    
    return final

def permutation(P_TABLE, PLAINTEXT):
    permutated_M = ""
    for index in P_TABLE:
        permutated_M += PLAINTEXT[int(index)-1]
    return permutated_M

def split48bits_in_half(binarybits):
    return binarybits[:24], binarybits[24:]

get_bin = lambda x, n: format(x, 'b').zfill(n)

def into_intArray(message: str):
    int_array = []
    mesg_array = list(message) 
    for i in mesg_array:
        int_array.append(ord(i))
    return int_array

def intList_to_binStr(message_list):
    binary = []
    for x in message_list: 
        binary.append(get_bin(x, 8))
    binary_str = ""
    for x in binary:
        binary_str+=x 
    return binary_str


def Encrypt(message,key):
    cipher = ""
    
    L,R = split48bits_in_half(message)
    
    for cnt in range(16):
        f_out = F(R, key[cnt])
        l_out = XOR(L, f_out)
        L , R = R, l_out

    return L+R
    
def Decrypt(message,key):
    cipher = ""
   
    L, R = split48bits_in_half(message)

    for cnt in range(15, -1, -1):
        f_out = F(L, key[cnt])
        l_out = XOR(f_out, R)
        L , R = l_out, L

    return L+R  


M = "alamak"
key = "askmvdapsec3wlmfs" 

message = intList_to_binStr(into_intArray(M)) 
print("Plaintext:  ", message)


binary_key = intList_to_binStr(into_intArray(key)) 
subkeys = generate_keys(binary_key)

ciphertext = Encrypt(message, subkeys) 
print("Encrypted:  ", ciphertext)
decrypted = Decrypt(ciphertext, subkeys) 
print("Decrypted:  ", decrypted)
print("XOR of message and decrypted message: ",XOR(message, decrypted))

Plaintext:   011000010110110001100001011011010110000101101011
Encrypted:   010000101001110011100111011011010011111000101111
Decrypted:   011000010110110001100001011011010110000101101011
XOR of message and decrypted message:  000000000000000000000000000000000000000000000000
